In [1]:
'''
Created on Nov 24, 2017

Purpose is to replicate results of Generating Adversarial Examples with Adversarial Networks, ICLR 2018.
This script implements the training process for the GAN model.
'''
import sys
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
from tensorflow.python.client import device_lib
import pickle
print(device_lib.list_local_devices())
import numpy as np
#from School.STAT946.Project impimport GAN
import keras
import GAN
from Losses import *
from keras.optimizers import SGD
import keras.backend as K
import tensorflow as tf
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = (500/12000)
set_session(tf.Session(config=config))
'''
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
# Assume that you have 12GB of GPU memory and want to allocate 500MB:
config.gpu_options.per_process_gpu_memory_fraction=(500/12000)
sess = tf.Session(config=config)'''
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                       logits=predicted)



[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10289415204807432219
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11332668621
locality {
  bus_id: 1
}
incarnation: 6631158176850942352
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:06:00.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 11330676327
locality {
  bus_id: 1
}
incarnation: 15407026887944723734
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:07:00.0"
, name: "/gpu:2"
device_type: "GPU"
memory_limit: 11330614068
locality {
  bus_id: 1
}
incarnation: 4996095157860593154
physical_device_desc: "device: 2, name: Tesla K80, pci bus id: 0000:0e:00.0"
, name: "/gpu:3"
device_type: "GPU"
memory_limit: 11328684032
locality {
  bus_id: 1
}
incarnation: 16782623752990443102
physical_device_desc: "device: 3, name: Tesla K80, pci bus id: 0000:0f:00.0"
]


Using TensorFlow backend.


In [2]:
def train_for_n(epochs=5000,batch_size=128):
    #from tqdm import tqdm
    
    for tk in range(epochs):
        if(tk==0):
            print("Entered the loop")
        # Make generative images
        real_image_batch = X[np.random.randint(0,X.shape[0],size=int(batch_size/2)),:,:,:]
        fake_image_inp = X[np.random.randint(0,X.shape[0],size=int(batch_size/2)),:,:,:]
        fake_image_batch = np.add(fake_image_inp,G.predict(fake_image_inp))

        # Train discriminator on generated images
        X_batch = np.concatenate((real_image_batch, fake_image_batch))
        y1 = np.zeros([batch_size,1])
        y1[0:int(batch_size/2),] = 1
        
        make_trainable(D,True)
        D.train_on_batch(X_batch,y1)
        #d_loss  = D.train_on_batch(X_batch,y1)
        #losses["d"][e]=d_loss

        #train Generator-Discriminator stack on input noise to non-generated output class
        sample_int=np.random.randint(0,X.shape[0],size=int(batch_size))
        fake_image_inp = X[sample_int,:,:,:]
        y_discrim = np.ones([batch_size,1])
        y_class=y[sample_int]
        y_hinge=np.zeros([batch_size,28,28,1])
        
        make_trainable(D,False)
        scalarloss=GAN.train_on_batch(fake_image_inp, [y_discrim,y_class,y_hinge])
        #g_loss = GAN.train_on_batch(noise_tr, y2 )
        if(tk%500==0):
            print("Epoch number:",tk,"; Loss",scalarloss)
         #losses["g"][e]=g_loss
    #GAN.save('GAN_CPristine100')
    #G.save('G_CPristine100')
    #D.save('D_CPristine100')
    #f.save('f_CPristine100')
    
    return



In [3]:
GAN, G, D, f= GAN.Define_GAN([28,28,1], 100, 10000,'B')
print("=||="*7)
print("=||="*7)
#f.summary()
(X,y),(_,_)=mnist.load_data()
X= np.divide(X,255)
X=X.reshape(X.shape[0],28,28,1)
y=to_categorical(y, num_classes=10)
make_trainable(f,False)
print('Model F\n')
f.summary()
#make_trainable(f2,False)


Phew
I'm here 111
Tensor("model_1_target:0", shape=(?, ?, ?, ?), dtype=float32)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 28, 28, 1)     0                                            
____________________________________________________________________________________________________
model_1 (Model)                  (None, 28, 28, 1)     85757       input_3[0][0]                    
____________________________________________________________________________________________________
add_5 (Add)                      (None, 28, 28, 1)     0           input_3[0][0]                    
                                                                   model_1[1][0]                    
____________________________________________________________________________________________________
model_2 (Mo

In [4]:
train_for_n(epochs=2000,batch_size=128)
GAN.save_weights('WhiteBox/B_Wbox_Ganwt3k')
G.save_weights('WhiteBox/B_Wbox_Gwt3k')
D.save_weights('WhiteBox/B_Wbox_Dwt3k')
f.save_weights('WhiteBox/B_Wbox_fwt3k')
print('Done')
#GAN.save('GAN_BPristine15000')
#G.save('G_BPristine15000')
#D.save('D_BPristine15000')
#f.save('f_BPristine15000')

Entered the loop
Epoch number: 0 ; Loss [18501.371, 0.57678562, 0.50421798, 1.8450373, 0.0078125, 0.671875, 0.75310874]
Epoch number: 500 ; Loss [95.327805, 0.19876346, 0.30905026, 0.0064224014, 1.0, 0.5, 0.99999958]
Epoch number: 1000 ; Loss [28.021957, 0.19256291, 0.27769381, 6.0014509e-06, 1.0, 0.4609375, 0.99999958]
Epoch number: 1500 ; Loss [46.326061, 0.19508506, 0.46130976, 0.0, 1.0, 0.6796875, 0.99999958]
Done


In [5]:

#This is the compile function for C. To be called "After freezing the weights"
#sgd= SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#f2.compile(loss=fn,optimizer=sgd,metrics=['accuracy'])
#f.summary()
#make_trainable(f2,False)

In [6]:
#GAN, G, D, f= GAN.Define_GAN([28,28,1], 1, 10000,'B')
'''GAN.load_weights('WhiteBox/B_Wbox_Ganwt2k')
G.load_weights('WhiteBox/B_Wbox_Gwt2k')
D.load_weights('WhiteBox/B_Wbox_Dwt2k')
f.load_weights('WhiteBox/B_Wbox_fwt2k')
make_trainable(f, False)'''

"GAN.load_weights('WhiteBox/B_Wbox_Ganwt2k')\nG.load_weights('WhiteBox/B_Wbox_Gwt2k')\nD.load_weights('WhiteBox/B_Wbox_Dwt2k')\nf.load_weights('WhiteBox/B_Wbox_fwt2k')\nmake_trainable(f, False)"